# SPM4 bonus material for Bill

Theme Song: La Di Da<br>
Artist: VUKOVI<br>
Album: VUKOVI<br>
Released: 2017

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
import matplotlib.patches as mp
from netCDF4 import Dataset
import warnings

from ar6.utils.h5 import *

In [2]:
# TODO - sort out figure sizing

pl.rcParams['figure.figsize'] = (18/2.54, 11/2.54)
pl.rcParams['font.size'] = 11
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.size'] = 0
pl.rcParams['xtick.top'] = True
pl.rcParams['figure.dpi'] = 150

In [3]:
results = load_dict_from_hdf5('../data_output_large/twolayer_SSPs.h5')

In [4]:
results['ssp245']['surface_temperature'].shape

(351, 2237)

In [5]:
results['ssp245']['surface_temperature'][269].mean()

1.2882961864594409

In [6]:
forcings = ['co2', 'ch4', 'aerosol']
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
for scenario in scenarios:
    for forcing in forcings:
        print(scenario, forcing, (results[scenario]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351]).mean())

ssp119 co2 1.184257818138223
ssp119 ch4 0.1479647406367096
ssp119 aerosol -0.16204737745274067
ssp126 co2 1.5144231930207652
ssp126 ch4 0.15700660225583174
ssp126 aerosol -0.16330764057886007
ssp245 co2 2.218355784976657
ssp245 ch4 0.2848954746587781
ssp245 aerosol -0.27867451649624025
ssp370 co2 2.9466805281763078
ssp370 ch4 0.4981462326525
ssp370 aerosol -0.5732443586633299
ssp585 co2 3.5989803009371477
ssp585 ch4 0.42262311099467836
ssp585 aerosol -0.327737943985368


In [7]:
(results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][268:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][0]).shape

(83, 2237)

In [8]:
forcings = ['co2', 'ch4', 'aerosol', 'other_ghg', 'other_anthro']

AR6_ecsforc = {}
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

base_period = slice(100,151)

for scenario in scenarios:
    AR6_ecsforc[scenario] = {}
    for forcing in forcings:
#        AR6_ecsforc[scenario][forcing] = np.zeros((83, 5))
        AR6_ecsforc[scenario][forcing] = np.percentile(
            (
                (results['%s_climuncert' % scenario]['surface_temperature'][268:351] - results['%s_climuncert' % scenario]['surface_temperature'][base_period].mean(axis=0))-
                (results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][268:351] - results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][base_period].mean(axis=0))
            ), (5,16,50,84,95), axis=1
        )
    AR6_ecsforc[scenario]['anthro'] = np.percentile(results['%s_climuncert' % scenario]['surface_temperature'][268:351] - results['%s_climuncert' % scenario]['surface_temperature'][base_period].mean(axis=0), (5,16,50,84,95), axis=1)

In [9]:
output = []
for scenario in scenarios:
    for forcing in ['co2', 'ch4', 'aerosol', 'other_ghg', 'other_anthro', 'anthro']:
        output.append([scenario, forcing]+[i for i in AR6_ecsforc[scenario][forcing][2,:]])

df = pd.DataFrame(output, columns=['scenario', 'forcing'] + list(range(2018,2101)))
df.to_csv('../data_output/spm4_extended_warming_contributions.csv', index=False)